In [0]:
import numpy as np
from operator import add

edgesRDD = sc.textFile('/FileStore/tables/comp4651-project/musae_squirrel_edges.csv', 8)
header = edgesRDD.first()
edgesRDD = (
  edgesRDD.filter(lambda line: line != header)
      .map(lambda line: tuple(map(int, line.split(','))))
      .distinct()
)

def join_list(list1, list2):
  if list1 is None:
    return list2
  if list2 is None:
    return list1
  else:
    return list1+list2

nodesRDD = (
  edgesRDD.groupByKey().mapValues(list)
    .fullOuterJoin(edgesRDD.map(lambda kv: (kv[1], kv[0])).groupByKey().mapValues(list))
    .mapValues(lambda v: join_list(v[0], v[1]))
    .mapValues(np.unique)
).cache()

numNodes = nodesRDD.count()

In [0]:
adjanceyDict = sc.broadcast(dict(nodesRDD.collect()))

In [0]:
def findClusterCoefficient(neighbors):
  numNeighbors = len(neighbors)
  maxEdges = numNeighbors * (numNeighbors - 1)
  if maxEdges == 0:
    return 0
  edgesBetweenNeighbors = 0
  for neighbor in neighbors:
      edgesBetweenNeighbors += len(np.intersect1d(neighbors, adjanceyDict.value[neighbor]))
  return min(1.0, edgesBetweenNeighbors/maxEdges)
  
clusterCoefficient = nodesRDD.mapValues(findClusterCoefficient).cache()

In [0]:
from pyspark.sql.functions import expr

ccDF = clusterCoefficient.toDF(['id', 'cc'])
ccStat = (
  ccDF.withColumn('group', expr('int(cc/0.02)'))
    .groupBy('group')
    .count()
    .selectExpr('group * 0.02 as cc','count as freq')
    .orderBy('cc')
    .withColumn('logC', expr('log10(cc)'))
    .withColumn('p', expr('freq/{}'.format(numNodes)))
    .withColumn('pdf', expr('p*50'))
    .withColumn('cdf', expr('sum(p) over (order by cc)'))
)

In [0]:
display(ccStat)

cc,freq,logC,p,pdf,cdf
null,156,null,0.029994231878484906,1.4997115939242454,0.029994231878484906
0.00,181,null,0.034800999807729285,1.7400499903864644,0.06479523168621419
0.02,29,-1.6989700043360187,0.005575850797923477,0.2787925398961738,0.07037108248413766
0.04,39,-1.3979400086720375,0.0074985579696212265,0.37492789848106134,0.07786964045375888
0.06,42,-1.2218487496163564,0.008075370121130552,0.4037685060565276,0.08594501057488943
0.08,64,-1.0969100130080565,0.012305325898865602,0.6152662949432801,0.09825033647375503
0.10,81,-1.0,0.015573928090751778,0.7786964045375889,0.11382426456450681
0.12,89,-0.9208187539523752,0.017112093828109978,0.8556046914054989,0.1309363583926168
0.14,86,-0.8538719643217619,0.016535281676600653,0.8267640838300326,0.14747164006921745
0.16,123,-0.7958800173440752,0.02364929821188233,1.1824649105941165,0.17112093828109978


In [0]:
clusterCoefficient.map(lambda kv: '{} {}'.format(kv[0], kv[1])).coalesce(1).saveAsTextFile('/FileStore/tables/comp4651-project/output1')